In [1]:
%load_ext line_profiler

In [2]:
from sklearn.datasets import make_blobs

from divik.cluster import KMeans, DunnDiviK
import divik.cluster._kmeans._core
import divik.cluster._kmeans._gap
import divik.cluster._kmeans._initialization
import divik.cluster._divik._dunn
import divik.score._sampled_gap
import divik.score._gap

In [3]:
X, y = make_blobs(n_samples=100000, n_features=100, centers=10, random_state=42)

In [4]:
kmeans = KMeans(n_clusters=10)

In [5]:
mdl = DunnDiviK(distance='euclidean', filter_type='auto', n_jobs=1, features_percentage=0.2, verbose=True)

# Generic timing

In [6]:
%lprun \
    -f divik.cluster._kmeans._initialization.PercentileInitialization.__call__ \
    -f divik.cluster._kmeans._core._KMeans.__call__ \
    kmeans.fit(X)

Timer unit: 1e-06 s

Total time: 6.46002 s
File: /app/divik/cluster/_kmeans/_core.py
Function: __call__ at line 111

Line #      Hits         Time  Per Hit   % Time  Line Contents
   111                                               def __call__(self, data: Data, number_of_clusters: int) \
   112                                                       -> Tuple[IntLabels, Centroids]:
   113         1         11.0     11.0      0.0          _validate_kmeans_input(data, number_of_clusters)
   114         1          1.0      1.0      0.0          if number_of_clusters == 1:
   115                                                       return np.zeros((data.shape[0], 1), dtype=int), \
   116                                                              np.mean(data, axis=0, keepdims=True)
   117         1          9.0      9.0      0.0          data = data.reshape(data.shape, order='C')
   118         1          1.0      1.0      0.0          if self.normalize_rows:
   119                      

In [7]:
%lprun \
    -f divik.cluster._divik._dunn.dunn_divik \
    -f divik.cluster._kmeans._gap.GAPSearch._fit_kmeans \
    -f divik.score._sampled_gap.sampled_gap \
    -f divik.score._sampled_gap._dispersion \
    mdl.fit(X)

  0%|          | 0/100000 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:23<00:00, 11.82s/it]
                                             
100%|██████████| 9/9 [00:01<00:00,  5.43it/s]
                                             
 50%|█████     | 1/2 [00:06<00:06,  6.15s/it]
                                             

 10%|█         | 10000/100000 [00:36<05:24, 277.50it/s]


 50%|█████     | 1/2 [00:04<00:04,  4.95s/it]
                                             

 20%|██        | 20000/100000 [00:43<03:39, 363.70it/s]

...


 50%|█████     | 1/2 [00:04<00:04,  4.51s/it]
                                             

 90%|█████████ | 90000/100000 [01:50<00:11, 904.03it/s]


 50%|█████     | 1/2 [00:11<00:11, 11.14s/it]
                                             

100%|██████████| 100000/100000 [02:06<00:00, 788.87it/s]


Timer unit: 1e-06 s

Total time: 126.747 s
File: /app/divik/cluster/_divik/_dunn.py
Function: dunn_divik at line 25

Line #      Hits         Time  Per Hit   % Time  Line Contents
    25                                           def dunn_divik(data: Data, selection: np.ndarray,
    26                                                          fast_kmeans: GAPSearch, full_kmeans: DunnSearch,
    27                                                          feature_selector: StatSelector,
    28                                                          minimal_size: int, rejection_size: int, report: DivikReporter) \
    29                                                   -> Optional[DivikResult]:
    30        11      70578.0   6416.2      0.1      subset = data[selection]
    31                                           
    32        11        160.0     14.5      0.0      if subset.shape[0] <= max(full_kmeans.max_clusters, minimal_size):
    33                                              

# Init timing

In [8]:
%lprun \
    -f divik.cluster._kmeans._initialization.PercentileInitialization.__call__ \
    -f divik.cluster._kmeans._initialization._find_residuals \
    kmeans.fit(X)

Timer unit: 1e-06 s

Total time: 0.964557 s
File: /app/divik/cluster/_kmeans/_initialization.py
Function: _find_residuals at line 37

Line #      Hits         Time  Per Hit   % Time  Line Contents
    37                                           def _find_residuals(data: Data) -> np.ndarray:
    38         1          2.0      2.0      0.0      features = data.T
    39         1          3.0      3.0      0.0      assumed_ys = features[0]
    40         1        606.0    606.0      0.1      modelled_xs = np.hstack([np.ones((data.shape[0], 1)),
    41         1      31259.0  31259.0      3.2                               features[1:].T])
    42         1     925684.0 925684.0     96.0      coefficients = _lstsq_numba(modelled_xs, assumed_ys)
    43         1       7000.0   7000.0      0.7      residuals = _residuals_numba(modelled_xs, assumed_ys, coefficients)
    44         1          3.0      3.0      0.0      return residuals

Total time: 1.09622 s
File: /app/divik/cluster/_kmeans/_in

In [9]:
%lprun \
    -f divik.cluster._kmeans._initialization.PercentileInitialization.__call__ \
    -f divik.cluster._kmeans._initialization._find_residuals \
    mdl.fit(X)

  0%|          | 0/100000 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:19<00:00, 10.69s/it]
                                             
100%|██████████| 9/9 [00:01<00:00,  5.34it/s]
                                             
 50%|█████     | 1/2 [00:06<00:06,  6.44s/it]
                                             

 10%|█         | 10000/100000 [00:31<04:45, 314.82it/s]


 50%|█████     | 1/2 [00:03<00:03,  3.17s/it]
                                             

 20%|██        | 20000/100000 [00:36<03:09, 421.67it/s]

...


 50%|█████     | 1/2 [00:04<00:04,  4.37s/it]
                                             

 90%|█████████ | 90000/100000 [01:37<00:10, 958.41it/s]


 50%|█████     | 1/2 [00:06<00:06,  6.80s/it]
                                             

100%|██████████| 100000/100000 [01:49<00:00, 909.41it/s]


Timer unit: 1e-06 s

Total time: 0.826141 s
File: /app/divik/cluster/_kmeans/_initialization.py
Function: _find_residuals at line 37

Line #      Hits         Time  Per Hit   % Time  Line Contents
    37                                           def _find_residuals(data: Data) -> np.ndarray:
    38       140        310.0      2.2      0.0      features = data.T
    39       140        318.0      2.3      0.0      assumed_ys = features[0]
    40       140       7099.0     50.7      0.9      modelled_xs = np.hstack([np.ones((data.shape[0], 1)),
    41       140      53914.0    385.1      6.5                               features[1:].T])
    42       140     706356.0   5045.4     85.5      coefficients = _lstsq_numba(modelled_xs, assumed_ys)
    43       140      56853.0    406.1      6.9      residuals = _residuals_numba(modelled_xs, assumed_ys, coefficients)
    44       140       1291.0      9.2      0.2      return residuals

Total time: 1.41629 s
File: /app/divik/cluster/_kmeans/_in